In [1]:
import re
import pandas as pd

In [2]:
def load_dictionary(filename):
    vndict = {}
    dict_df = pd.read_csv(filename, encoding='utf-8')

    for index, row in dict_df.iterrows():
        word = row['Word'].strip()
        score = [float(row['Score'])]
        vndict[word] = score
    return vndict

def load_special_word(filename):
    special_word = {}
    dict_df = pd.read_csv(filename, encoding='utf-8')

    for index, row in dict_df.iterrows():
        word = row['Word'].strip()
        special_word[word] = word
    return special_word

In [3]:
def final_score(tokens):
    score = 0
    for i in range(len(tokens)):
        word = tokens[i]
        if word in vndict:
            _score = vndict[word][0]
            if i-1 >= 0 and tokens[i-1] in oppositeWord_dict:
                _score *= -1
            elif i-2 >= 0 and tokens[i-2] in oppositeWord_dict:
                _score *= -1
            score += _score
    return score

"""
def final_score(tokens):
    score = 0
    for i in range(len(tokens)):
        word = tokens[i]
        if word in vndict:
            score += vndict[word][0]
    return score
"""

def define_sentiment(score):
    if score > 0:
        return 'positive'
    else:
        return 'negative'

In [4]:
def predict_dataset(input_file, output_file):
    try:
        data = pd.read_csv(input_file, encoding='utf-8')
    except FileNotFoundError:
        print("File input không tồn tài!")
        return

    print("Input length:", len(data))
    rows = []

    for index, row in data.iterrows():
        tokens = eval(row['Tokens'])
        score = final_score(tokens)
        pre_sentiment = define_sentiment(score)
        rows.append([score, pre_sentiment])

    df = pd.DataFrame(rows, columns=['Score', 'pre_Sentiment'])
    existing_df = pd.read_csv(input_file, encoding='utf-8')
    combined_df = pd.concat([df, existing_df], axis=1)
    combined_df.to_csv(output_file, index=False, encoding='utf-8')

vndict = load_dictionary('/content/drive/MyDrive/KLTN/dictionary/VN_UIT_dict.csv')
oppositeWord_dict = load_special_word('/content/drive/MyDrive/KLTN/dictionary/opposite_word.csv')

In [5]:
vndict = load_dictionary('/content/drive/MyDrive/KLTN/dictionary/VN_UIT_dict.csv')
oppositeWord_dict = load_special_word('/content/drive/MyDrive/KLTN/dictionary/opposite_word.csv')


In [6]:
predict_dataset('/content/drive/MyDrive/KLTN/output/UIT_Final.csv', '/content/drive/MyDrive/KLTN/output/UIT_Final_score.csv')

Input length: 14876


In [8]:
def compare_sentiments(input_file):
    # Đọc dữ liệu từ tệp CSV
    data = pd.read_csv(input_file, encoding='utf-8')

    # Tính số lượng dòng giống nhau giữa hai cột
    total_rows = len(data)
    matching_rows = len(data[data['pre_Sentiment'] == data['Sentiment']])

    # Tính phần trăm giống nhau
    percentage_matching = (matching_rows / total_rows) * 100
    return percentage_matching

#input_file = '/content/drive/MyDrive/KLTN/output/UIT_Final_score.csv'
input_file = '/content/drive/MyDrive/KLTN/output/UIT_Final_score_no_oppositeWord.csv'
percentage_matching = compare_sentiments(input_file)
print(f"Phần trăm giống nhau giữa 'pre_Sentiment' và 'Sentiment' là {percentage_matching:.2f}%")

Phần trăm giống nhau giữa 'pre_Sentiment' và 'Sentiment' là 82.03%
